In [ ]:
#Use this code after running the minimap2 using the code pooled_GPCR_run_minimap2.sh
#check how many input reads in the beginning
import os
from Bio import SeqIO

read_summaryD={}
sampleL=[]

stitch_orderL=["Stitch1_replicate", "Stitch2_replicate", "Stitch3_replicate", "Stitch4_replicate"]
repeatL=["1-1","1-2","2-1","2-2"]
#1st stitch

for i in range (0,4):
    for repeat in repeatL:
        sampleL.append(stitch_orderL[i]+repeat)
        file_path=stitch_orderL[i]+repeat+".fastq"
        count = sum(1 for record in SeqIO.parse(file_path, "fastq"))
        read_summaryD[stitch_orderL[i]+repeat]=count
        
print(read_summaryD)

{'Stitch1_replicate1-1': 2144, 'Stitch1_replicate1-2': 1077, 'Stitch1_replicate2-1': 1485, 'Stitch1_replicate2-2': 2320, 'Stitch2_replicate1-1': 418, 'Stitch2_replicate1-2': 1604, 'Stitch2_replicate2-1': 3015, 'Stitch2_replicate2-2': 896, 'Stitch3_replicate1-1': 84716, 'Stitch3_replicate1-2': 76016, 'Stitch3_replicate2-1': 55020, 'Stitch3_replicate2-2': 43223, 'Stitch4_replicate1-1': 299149, 'Stitch4_replicate1-2': 253614, 'Stitch4_replicate2-1': 684302, 'Stitch4_replicate2-2': 132052}


In [74]:
#check how many reads mapped to the single ref.
#use this to generate Table S8. Read coverage and number of GPCR designs detected at various steps of filtering for the pooled GPCR library assembly data.

#run mimimap2 to get this paf file
#alalysis based on the alignment results and also compare to the known posistion GPCR

import os, re

alignD={}
assignD={}
mappedD={}
removeD={}
#path = '/Users/po-hsianghung/Documents/GPCR/pool_assay_analysis/raw_demuxed_file/merged_reads/'

out1=open("reads_mapped_summary.xls",'w')
out2=open("GPCR_assemble_summary.xls",'w')
out1.write("barcode"+"\t"+"total_reads"+"\t"+"total_mapped_reads"+"\t"+"total_unique_mapped_reads"+"\n")

#for test purpose, using few samples
#sampleL=['A1-1','B1-1','C1-1','D1-1']

for n_barcode in sampleL:
    alignD[n_barcode]={}
    mappedD[n_barcode]={}
    removeD[n_barcode]={}
    n_total=0 #this is mappable reads. It is not equal to the original reads
    removeD[n_barcode]=[]
    file2=open("paf/"+n_barcode+".paf")
    #record the best assign into alignD, if there are same socre with multiple hits - go to removeD
    for line in file2.readlines():
        line=line.strip().split("\t")
        read_name=line[0]
        read_length=line[1]
        ref_name=line[5]
        ref_length=line[6]
        n_mapped=int(line[9]) #number of residule mapped
        if read_name in alignD[n_barcode].keys():
            if alignD[n_barcode][read_name][2] < n_mapped:
                alignD[n_barcode][read_name]=[read_length,ref_name, n_mapped]
            elif alignD[n_barcode][read_name][2] == n_mapped: #the same, can not distinguish
                if read_name not in removeD[n_barcode]:
                    removeD[n_barcode].append(read_name)
        else:
            n_total=n_total+1
            alignD[n_barcode][read_name]=[read_length,ref_name, n_mapped]
    #count unique mapped reads into reference [mappedD]
    for read_name in list(alignD[n_barcode].keys()):
        if read_name not in removeD[n_barcode]: 
            ref_name=alignD[n_barcode][read_name][1]
            if ref_name in mappedD[n_barcode].keys():
                mappedD[n_barcode][ref_name]=mappedD[n_barcode][ref_name]+1
            else:
                mappedD[n_barcode][ref_name]=1
     
    out1.write(str(n_barcode)+"\t"+str(read_summaryD[n_barcode])+"\t"+str(n_total)+"\t"+str(n_total-len(removeD[n_barcode]))+"\n")




gpcr=["A", "B", "C", "D","E","F"]
#stitch_orderL=["A", "B", "C", "D"]
gpcrL=[]

for a in gpcr:
   gpcrL.append(a)

for a in gpcr:
    for b in gpcr:
        gpcrL.append(a+b)

for a in gpcr:
    for b in gpcr:
        for c in gpcr:
            gpcrL.append(a+b+c)

for a in gpcr:
    for b in gpcr:
        for c in gpcr:            
            for d in gpcr:
                gpcrL.append(a+b+c+d)



#output the out2 file - GPCR reads information


out2.write("GPCR"+"\t"+"stitch"+"\t")
for sample in sampleL:
    out2.write(sample+"\t")
out2.write("\n")

resultD={}
for ref in gpcrL:
    resultD[ref]=['.']*len(sampleL)
    for sample in sampleL:
        
        if ref in mappedD[sample].keys():
            resultD[ref][sampleL.index(sample)]=str(mappedD[sample][ref])
        else:
            resultD[ref][sampleL.index(sample)]="0"

for ref in gpcrL:
    out2.write(ref+"\t"+str(len(ref))+"\t")
    for num in resultD[ref]:
        out2.write(num+"\t")
    out2.write("\n")


out2.close()
out1.close()


In [76]:
#for plot
#first, add expected values and parnet information for making plots
#use this to generate Table S10. Data analysis summary for the pooled GPCR assembly. 

file2=open("GPCR_assemble_summary.xls")

out1=open("pool_summary_forplot.csv",'w')
out1.write("unique id"+","+"GPCR"+","+"parent"+","+"#stitch"+","+"1st"+","+"2nd"+","+"3rd"+","+"4th"+","+"R_1-1"+","+"R_1-2"+","+"R_2-1"+","+"R_2-2"+
",frq_1-1,frq_1-2,frq_2-1,frq_2-2,exp_1-1,exp_frq_1-1,exp_1-2,exp_frq_1-2,exp_2-1,exp_frq_2-1,exp_2-2" +   ",exp_frq_2-2,"+"\n")


#for lineage plot
#add expected values and parnet information for making plots
out2=open("pool_summary_forplot_lineage.csv",'w')
out2.write("unique id"+","+"GPCR"+","+"parent"+","+"#stitch"+","+"1st"+","+"2nd"+","+"3rd"+","+"4th"+","+"R_1-1"+","+"R_1-2"+","+"R_2-1"+","+"R_2-2"+
",frq_1-1,frq_1-2,frq_2-1,frq_2-2,exp_1-1,exp_frq_1-1,exp_1-2,exp_frq_1-2,exp_2-1,exp_frq_2-1,exp_2-2" +   ",exp_frq_2-2,"+"\n")


#input the 0 stitch - recipeint only for lineage tracing.
out2.write("-1"+","+"R"+","+"NA"+","+"0"+","+"NA"+","+"NA"+","+"NA"+","+"NA"+","+"1"+","+"1"+","+"1"+","+"1"+
",1,1,1,1,1,1,1,1,1,1,1,1,"+"\n")


resultD={}

#GPCRL=gpcrL

for line in file2.readlines():
    if line.startswith("GPCR"):
        line=line.strip().split("\t")
        head=line
    else:
        line=line.strip().split("\t")
        #resultD[gpcrL.index(line[0])]=[line[0], gpcrL.index(line[0][0:3]) ,len(line[0])] + list(line[0])+["NA"]*(4-len(line[0]))+line[1:5]
        if len(line[0])==1: #1st stitch
            resultD[gpcrL.index(line[0])]=[line[0],-1,len(line[0])] + list(line[0])+["NA"]*(4-len(line[0]))+line[2:6]
        elif len(line[0])==2: #2nd stitch
            resultD[gpcrL.index(line[0])]=[line[0],  gpcrL.index(line[0][0]) ,len(line[0])] + list(line[0])+["NA"]*(4-len(line[0]))+line[6:10]
        elif len(line[0])==3: #3nd stitch
            resultD[gpcrL.index(line[0])]=[line[0], gpcrL.index(line[0][0:2])   ,len(line[0])] + list(line[0])+["NA"]*(4-len(line[0]))+line[10:14]
        elif len(line[0])==4: #4th stitch
            resultD[gpcrL.index(line[0])]=[line[0], gpcrL.index(line[0][0:3]) ,len(line[0])] + list(line[0])+["NA"]*(4-len(line[0]))+line[14:18]
        else:
            print ("error: only 4 stitch, ", line)



totalD={}
for i in range (0, len(resultD)):
    stitch=resultD[i][2]
    if stitch in totalD.keys():
        for j in range (0,4):
            totalD[stitch][j]=int(totalD[stitch][j])+int(resultD[i][7:11][j])
    else:
        totalD[stitch]=resultD[i][7:11]

#calculate the frequency for each stitch
for i in range (0, len(resultD)):
    stitch=resultD[i][2]
    n_total=totalD[stitch]
    for j in range (0, len(resultD[i][7:11])):
        #print (resultD[i][7+j] , n_total[j])
        resultD[i].append(     int(resultD[i][7+j])     /n_total[j] )
#calculate the expected value
for n in range (0, len(resultD)):
    stitch=resultD[n][2]
    n_total=totalD[stitch]
    parent=resultD[n][1]
    exp_frq=float(1)/6
    
    if parent==-1:
        p_frq=[1,1,1,1]
    else:
        p_frq=resultD[parent][11:15] #parental real frq.

    for j in range (0, len(p_frq)):
        resultD[n].append(p_frq[j] * exp_frq * n_total[j])
        resultD[n].append(p_frq[j] * exp_frq)

for i in range (0, len(resultD)):
    out1.write(str(i)+",")
    out2.write(str(i)+",")
    for item in resultD[i]:
        out1.write(str(item)+",")
        out2.write(str(item)+",")
    out1.write("\n")
    out2.write("\n")

out1.close()    
out2.close()  

In [ ]:
#identify missing GPCRs, whether the missing is due to the parental GPCR is failed assembled in previous round
# use this to generate Table S9. Table of missed GPCR variants. 
file5=open("pool_summary_forplot_lineage.csv")

out1=open("missing.xls",'w')


idD={}
for line in file5.readlines():
    if not line.startswith("unique"):
        line=line.strip().split(",")
        idD[line[0]]=line[1]



missD={}

repeatL=["1-1","1-2","2-1","2-2"]

countD={}


file5=open("pool_summary_forplot_lineage.csv")
for line in file5.readlines():
    if not line.startswith("unique"):
        line=line.strip().split(",")
        n_stitch=int(line[3])
        n_countL=line[8:12]
        if n_stitch!=0:
            if n_stitch in countD.keys():
                for i in range (0, 4):
                    if int(n_countL[i])!=0:
                        countD[n_stitch][i]=countD[n_stitch][i]+1
                    else:
                        #print (line[1],n_stitch,repeatL[i])
                        #out1.write(line[0]+"\t"+line[2]+"\t"+line[1]+"\t"+str(n_stitch)+"\t"+repeatL[i]+"\n")
                        if repeatL[i] in missD.keys():
                            if n_stitch in missD[repeatL[i]].keys():
                                if idD[line[2]] in missD[repeatL[i]][n_stitch].keys():
                                    missD[repeatL[i]][n_stitch][idD[line[2]]].append(line[1])
                                else:
                                    missD[repeatL[i]][n_stitch][idD[line[2]]]=[line[1]]

                            else:
                                missD[repeatL[i]][n_stitch]={}
                                missD[repeatL[i]][n_stitch][idD[line[2]]]=[line[1]]                                
                        else:
                            missD[repeatL[i]]={}
                            missD[repeatL[i]][n_stitch]={}
                            missD[repeatL[i]][n_stitch][idD[line[2]]]=[line[1]]
                            #print ('1')

            else:
                countD[n_stitch]=[0,0,0,0]
                for i in range (0, 4):
                    if int(n_countL[i])!=0:
                        countD[n_stitch][i]=countD[n_stitch][i]+1


#print (countD)
#print (missD)


out1.write("repeat"+"\t"+"stitch"+"\t"+"parent"+"\t"+"# of missed child"+"\t"+"missed child"+"\n")
for repeat in repeatL:
    #for stitch in sorted(list(missD[repeat].keys())):
    for stitch in list(missD[repeat].keys()):
        for parent in missD[repeat][stitch].keys():
            out1.write(repeat+"\t"+str(stitch)+"\t"+parent+"\t"+str(len(missD[repeat][stitch][parent]))+"\t")
            for item in missD[repeat][stitch][parent]:
                out1.write(item+"\t")
            out1.write("\n")      

out1.close()



In [ ]:
#identify unqiue hits
file5=open("pool_summary_forplot_lineage.csv")
uniqueD={}
for line in file5.readlines():
    if not line.startswith("unique"):
        line=line.strip().split(",")
        n_stitch=int(line[3])
        n_countL=line[8:12]
        variants=line[1]
        if n_stitch!=0:
            if n_stitch in uniqueD.keys():
                for i in range (0, 4):
                    if int(n_countL[i])!=0:
                        if variants not in uniqueD[n_stitch]:
                            uniqueD[n_stitch].append(variants)
                    
            else:
                uniqueD[n_stitch]=[]
                for i in range (0, 4):
                    if int(n_countL[i])!=0:
                        if variants not in uniqueD[n_stitch]:
                            uniqueD[n_stitch].append(variants)
                    

for i in range (1,5):
    print (i, len(uniqueD[i]))
